# Kausalität (Vorhersagbarkeit) zwischen den Handelsdaten Import, Export, BIP und den Militärausgaben mit Granger-Causality-Test prüfen

In [23]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
import matplotlib.pyplot as plt

## 1. Differenzierung der Daten

In [24]:
military_df = pd.read_csv("data/military_expenditure.csv", skiprows=4)
gdp_df = pd.read_csv("data/GDP.csv", skiprows=4)
imports_df = pd.read_csv("data/imports.csv", skiprows=4)
exports_df = pd.read_csv("data/exports.csv", skiprows=4)

years = [str(year) for year in range(2000, 2024)]

military_data = military_df[['Country Name'] + years]
imports_data = imports_df[['Country Name'] + years]
gdp_data = gdp_df[['Country Name'] + years]
exports_data = exports_df[['Country Name'] + years]

# differentiate data to achieve stationary
countries = sorted(set(military_data['Country Name']) & set(gdp_data['Country Name']) & set(imports_data['Country Name']) & set(exports_data['Country Name']))

def differ(df):
    df_out = df.copy()
    for country in countries:
        row = df[df['Country Name'] == country].copy()
        row = pd.melt(
            row,
            id_vars=['Country Name'],
            var_name='Year',
            value_name='Money',
            value_vars=[str(year) for year in years]
        )
        row["Money"] = row["Money"].diff()
        row['Country Name'] = country
        row = row.pivot(index="Country Name", columns="Year", values="Money")
        row.columns.name = None
        row = row.reset_index()

        for col in row.columns:
            if col in df_out.columns:
                df_out.loc[df_out["Country Name"] == country, col] = row.iloc[0][col]
    return df_out

military_data = differ(military_data)
imports_data = differ(imports_data)
exports_data = differ(exports_data)
gdp_data = differ(gdp_data)

Die Daten werden zunächst differenziert, um stationäre Zeitreihen zu erhalten. Dies ist notwendig, da der Granger-Causality-Test nur auf stationären Daten anwendbar ist.

## 2. Granger-Causality-Test für alle Staaten

In [40]:
years = [str(year) for year in range(2001, 2024)]

military_data = military_data[['Country Name'] + years]
imports_data = imports_data[['Country Name'] + years]
gdp_data = gdp_data[['Country Name'] + years]
exports_data = exports_data[['Country Name'] + years]

# merge datasets on country names
merged_df = military_data.merge(imports_data, on='Country Name', suffixes=('_military', '_imports'))
merged_df = merged_df.merge(gdp_data, on='Country Name')
merged_df.rename(columns={year: f"{year}_gdp" for year in years}, inplace=True)
merged_df = merged_df.merge(exports_data, on='Country Name')
merged_df.rename(columns={year: f"{year}_exports" for year in years}, inplace=True)

granger_results = []

for index, row in merged_df.iterrows():
    country = row['Country Name']
    military = row[[f'{year}_military' for year in years]].astype(float)
    imports = row[[f'{year}_imports' for year in years]].astype(float)
    gdp = row[[f'{year}_gdp' for year in years]].astype(float)
    exports = row[[f'{year}_exports' for year in years]].astype(float)

    if military.isnull().any() or imports.isnull().any() or gdp.isnull().any():
        print("Skip " + country + " because of missing data")
        continue

    try:
        data = pd.DataFrame({"imports": imports.values, "military": military.values})
        # Granger test: does military spending help predict imports?
        test_result = grangercausalitytests(data, maxlag=3)
        for lag in range(1, 4):
            p_value = round(test_result[lag][0]['ssr_ftest'][1], 4)
            granger_results.append({
                'Country': country,
                'KPI': 'Import',
                'Lag': lag,
                'P-Value': p_value
            })
    except Exception:
        pass

    try:
        data = pd.DataFrame({"exports": exports.values, "military": military.values})
        # Granger test: does military spending help predict exports?
        test_result = grangercausalitytests(data, maxlag=3)
        for lag in range(1, 4):
            p_value = round(test_result[lag][0]['ssr_ftest'][1], 4)
            granger_results.append({
                'Country': country,
                'KPI': 'Export',
                'Lag': lag,
                'P-Value': p_value
            })
    except Exception:
        pass

    try:
        data = pd.DataFrame({"gdp": gdp.values, "military": military.values})
        # Granger test: does military spending help predict gdp?
        test_result = grangercausalitytests(data, maxlag=3)
        for lag in range(1, 4):
            p_value = round(test_result[lag][0]['ssr_ftest'][1], 4)
            granger_results.append({
                'Country': country,
                'KPI': 'GDP',
                'Lag': lag,
                'P-Value': p_value
            })
    except Exception:
        pass

granger_df = pd.DataFrame(granger_results)

# print results
print(granger_df.sort_values(by='P-Value').head(20))
print(granger_df[granger_df["KPI"] == "Export"].head(20))

Skip Aruba because of missing data

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0296  , p=0.8652  , df_denom=19, df_num=1
ssr based chi2 test:   chi2=0.0343  , p=0.8531  , df=1
likelihood ratio test: chi2=0.0343  , p=0.8531  , df=1
parameter F test:         F=0.0296  , p=0.8652  , df_denom=19, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.4338  , p=0.6554  , df_denom=16, df_num=2
ssr based chi2 test:   chi2=1.1388  , p=0.5659  , df=2
likelihood ratio test: chi2=1.1090  , p=0.5744  , df=2
parameter F test:         F=0.4338  , p=0.6554  , df_denom=16, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.6007  , p=0.2372  , df_denom=13, df_num=3
ssr based chi2 test:   chi2=7.3876  , p=0.0605  , df=3
likelihood ratio test: chi2=6.2872  , p=0.0984  , df=3
parameter F test:         F=1.6007  , p=0.2372  , df_denom=13, df_num=3

Granger Causality
number of lags (no zero) 1
ssr based F test:  

### Granger Kausalität: Militärausgaben -> Import% Anteil an GDP

In [41]:
import pandas as pd
import pycountry
import plotly.express as px

df_map = granger_df[granger_df["KPI"] == "Import"]
df_map = df_map.loc[df_map.groupby('Country')['P-Value'].idxmin()]
print(df_map.head(20))

# world map
fig = px.choropleth(
    df_map,
    locations='Country',
    locationmode='country names',
    color='P-Value',
    hover_name='Country',
    hover_data=['Lag'],
    color_continuous_scale='OrRd',
    projection='natural earth',
    title='Granger Kausalität: Militärausgaben -> Import% des GDP'
)

fig.update_geos(showcoastlines=True, showland=True, fitbounds='locations')
fig.update_layout(margin={'r':0,'t':40,'l':0,'b':0})
fig.show()

                         Country     KPI  Lag  P-Value
2    Africa Eastern and Southern  Import    3   0.2372
11                       Albania  Import    3   0.4981
286                      Algeria  Import    2   0.7051
19                    Arab World  Import    2   0.1992
26                     Argentina  Import    3   0.5347
33                       Armenia  Import    1   0.0394
42                     Australia  Import    1   0.0002
51                       Austria  Import    1   0.1939
61                    Azerbaijan  Import    2   0.0257
105                      Bahrain  Import    1   0.2558
89                    Bangladesh  Import    3   0.0113
114                      Belarus  Import    1   0.1742
70                       Belgium  Import    2   0.6395
123                       Belize  Import    1   0.3819
132                      Bolivia  Import    1   0.4404
160                     Botswana  Import    2   0.5585
141                       Brazil  Import    1   0.2639
151       

Die meisten Länder haben P-Werte > 0.05 (z. B. Algerien mit p = 0.7051), was darauf hindeutet, dass vergangene Militärhaushaltsdaten ihre Importentwicklung nicht besser vorhersagen.
Einzig die USA zeigen mit p = 0.0284 (Lag 2) einen signifikanten Effekt, da ein p-Wert < 0.05 bedeutet, dass die Lags des Militärbudgets die Prognose der Importe statistisch signifikant verbessern.

### Granger Kausalität: Militärausgaben -> Export% Anteil an GDP

In [42]:
import pandas as pd
import pycountry
import plotly.express as px

df_map = granger_df[granger_df["KPI"] == "Export"]
df_map = df_map.loc[df_map.groupby('Country')['P-Value'].idxmin()]
print(df_map.head(20))

# world map
fig = px.choropleth(
    df_map,
    locations='Country',
    locationmode='country names',
    color='P-Value',
    hover_name='Country',
    hover_data=['Lag'],
    color_continuous_scale='OrRd',
    projection='natural earth',
    title='Granger Kausalität: Militärausgaben -> Export% des GDP'
)

fig.update_geos(showcoastlines=True, showland=True, fitbounds='locations')
fig.update_layout(margin={'r':0,'t':40,'l':0,'b':0})
fig.show()

                         Country     KPI  Lag  P-Value
5    Africa Eastern and Southern  Export    3   0.4646
14                       Albania  Export    3   0.0955
288                      Algeria  Export    1   0.3084
29                     Argentina  Export    3   0.2295
36                       Armenia  Export    1   0.0185
45                     Australia  Export    1   0.0000
54                       Austria  Export    1   0.3606
64                    Azerbaijan  Export    2   0.3458
108                      Bahrain  Export    1   0.0749
92                    Bangladesh  Export    3   0.0080
117                      Belarus  Export    1   0.4797
73                       Belgium  Export    2   0.6217
126                       Belize  Export    1   0.1749
136                      Bolivia  Export    2   0.8557
164                     Botswana  Export    3   0.1862
144                       Brazil  Export    1   0.6983
153            Brunei Darussalam  Export    1   0.4715
99        

Die meisten Länder weisen p-Werte oberhalb des in der Statistik üblichen Signifikanzniveaus von 0.05 auf, was darauf hinweist, dass vergangene Militärbudgetdaten ihre Exportentwicklung nicht signifikant vorhersagen.
Bei Armenien (p = 0.0185), dem Vereinigten Königreich (p = 0.0410) und den USA (p = 0.0038) liegen die p-Werte jedoch unter 0.05, sodass in diesen Fällen von einer signifikanten Granger-Kausalität ausgegangen werden kann

### Granger Kausalität: Militärausgaben -> GDP

In [43]:
import pandas as pd
import pycountry
import plotly.express as px

df_map = granger_df[granger_df["KPI"] == "GDP"]
df_map = df_map.loc[df_map.groupby('Country')['P-Value'].idxmin()]
print(df_map.head(20))

# world map
fig = px.choropleth(
    df_map,
    locations='Country',
    locationmode='country names',
    color='P-Value',
    hover_name='Country',
    hover_data=['Lag'],
    color_continuous_scale='OrRd',
    projection='natural earth',
    title='Granger Kausalität: Militärausgaben -> GDP'
)

fig.update_geos(showcoastlines=True, showland=True, fitbounds='locations')
fig.update_layout(margin={'r':0,'t':40,'l':0,'b':0})
fig.show()

                         Country  KPI  Lag  P-Value
8    Africa Eastern and Southern  GDP    3   0.5083
15                       Albania  GDP    1   0.7199
291                      Algeria  GDP    1   0.5663
22                    Arab World  GDP    2   0.0804
31                     Argentina  GDP    2   0.1993
39                       Armenia  GDP    1   0.0351
48                     Australia  GDP    1   0.0014
57                       Austria  GDP    1   0.3002
67                    Azerbaijan  GDP    2   0.1453
111                      Bahrain  GDP    1   0.3066
95                    Bangladesh  GDP    3   0.0012
120                      Belarus  GDP    1   0.0413
75                       Belgium  GDP    1   0.3727
129                       Belize  GDP    1   0.4161
138                      Bolivia  GDP    1   0.8176
166                     Botswana  GDP    2   0.0240
147                       Brazil  GDP    1   0.5027
156            Brunei Darussalam  GDP    1   0.4123
102         

Die meisten Länder weisen P-Werte über 0.05 auf (z. B. Albanien p = 0.7199; Algerien p = 0.5663), sodass vergangene Militärhaushaltswerte ihre BIP-Entwicklung nicht signifikant vorhersagen.
Lediglich „Upper middle income“ (p = 0.0349) und die World als Ganzes (p = 0.0459) liegen unter der 0.05-Schwelle und zeigen einen signifikanten Granger-Effekt.